In [31]:
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification
from transformers import XLNetTokenizer, TFXLNetForSequenceClassification
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
# tokenizer = AlbertTokenizer.from_pretrained("albert-base-v1")
# tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
# tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

In [33]:
# example to training dataset
examples = [
    {"text": "I loved the movie", "label": 1},
    {"text": "I hated the movie", "label": 0}
]

In [34]:
# tokenization and data preparation
def encode_example(example):
    encoding = tokenizer.encode_plus(
        example['text'],
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        return_attention_mask=True,
        truncation=True,
        return_tensors='tf'
    )
    return {'input_ids': encoding['input_ids'][0], 'attention_mask': encoding['attention_mask'][0]}, example['label']

In [35]:
# convert examples to tensorFlow dataset
dataset = tf.data.Dataset.from_generator(
    lambda: (encode_example(ex) for ex in examples),
    output_signature=(
        {'input_ids': tf.TensorSpec(shape=(128,), dtype=tf.int32),
         'attention_mask': tf.TensorSpec(shape=(128,), dtype=tf.int32)},
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

In [36]:
# batching
dataset = dataset.batch(2)

In [37]:
# loading the five models here
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
# model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
# model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
# model = TFAlbertForSequenceClassification.from_pretrained("albert-base-v1")
# model = TFXLNetForSequenceClassification.from_pretrained("xlnet-base-cased")
# model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
# training and params
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [39]:
# training
model.fit(dataset, epochs=100)

Epoch 1/100
1/1 [==============================] - 8s 8s/step - loss: 0.7143 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 0.6380 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 1s 1s/step - loss: 0.8182 - accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 1s 860ms/step - loss: 0.6753 - accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 1s 806ms/step - loss: 0.7047 - accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 1s 888ms/step - loss: 0.8530 - accuracy: 0.5000
Epoch 7/100
1/1 [==============================] - 1s 872ms/step - loss: 0.4997 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 1s 805ms/step - loss: 0.5165 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 1s 850ms/step - loss: 0.4500 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 1s 1s/step - loss: 0.3436 - accuracy: 1.0000
Epoch 11/100
1/1 

In [40]:
# testing data
test_examples = [
    "This is the best movie I have ever seen!",
    "Absolutely terrible! I will never watch this again.",
    "It was okay, not great but not bad either.",
    "you suck man",
    "this is so crazy and terrible to look at"
]

In [41]:
# preprocessing
test_encodings = tokenizer(test_examples, truncation=True, padding=True, max_length=128, return_tensors='tf')

In [42]:
# creating a tf dataset
test_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']},
    tf.constant([0] * len(test_examples))
))
test_dataset = test_dataset.batch(3)

In [43]:
# prediction
predictions = model.predict(test_dataset)

2/2 [==============================] - 2s 38ms/step


In [44]:
# decoding predictions
predicted_labels = np.argmax(predictions.logits, axis=1)
sentiments = ["Positive" if label == 1 else "Negative" for label in predicted_labels]

In [45]:
# logging sentiment values
for sentence, sentiment in zip(test_examples, sentiments):
    print(f"sentence: \"{sentence}\" - sentiment: {sentiment}")

sentence: "This is the best movie I have ever seen!" - sentiment: Positive
sentence: "Absolutely terrible! I will never watch this again." - sentiment: Positive
sentence: "It was okay, not great but not bad either." - sentiment: Positive
sentence: "you suck man" - sentiment: Positive
sentence: "this is so crazy and terrible to look at" - sentiment: Positive
